In [1]:
import pandas as pd
import numpy as np
from conct_db import enginex
import calendar, datetime
from datetime import date, datetime
from sqlalchemy.types import DATE

Conect succesfull


In [2]:
periodo = datetime.now()
mes = periodo.month
# 
periodo_real = date(periodo.year, periodo.month,1)
diccionario = {'Periodo':[periodo_real]}
df_periodo = pd.DataFrame(diccionario)
df_periodo['mes'] = pd.DatetimeIndex(df_periodo['Periodo']).month
df_periodo['Month'] = pd.to_datetime(df_periodo['mes'], format='%m').dt.month_name()
nombre_mes = str(df_periodo['Month'].values[0])
periodo_query = str(df_periodo.Periodo.values[0])[:10]
periodo_query
nombre_mes


'October'

In [3]:
df1 = pd.read_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Desafios_{}.xlsx'.format(nombre_mes), skipfooter=3)
df1 = df1[(df1['Distribuidor'] != 'Total') & (df1['ACCOUNT_ID'] != 'Total')]
df1['Periodo'] = pd.to_datetime(date(periodo.year,mes,1))
df1['COD_CLIENTE'] = df1.ACCOUNT_ID.str.extract('(0{2,}\d+)')
df1['COD_CLIENTE'] = df1['COD_CLIENTE'].astype('Int64')
df1.rename({'Código Distribuidor':'COD_DISTRI'}, axis=1, inplace=True)
df1.drop(['Cartera','ACCOUNT_ID', 'Distribuidor'], axis=1, inplace=True)
df1 = df1[['Periodo','COD_DISTRI', 'COD_CLIENTE', 'Visualizaron', 'Aceptaron', '%Aceptaron', 'Completaron', '%Completaron']]

El siguiente codigo extrae los ultimos numeros despues del agregado de ceros intermedio. falta pulirlo  
    
    df1['COD_CLIENTE'] = df1.ACCOUNT_ID.str.extract('(0{2,}\d+)')
    
    df1['COD_CLIENTE'] = df1.COD_CLIENTE.str.extract('([^0{,}]\d+(?:.\d+)?)') 

Para extraer numeros de un texto/numero utilizar la expresion regular 
    
    ('(\d+(?:\.\d+)?)')
    



In [4]:
# df1[pd.isna(df1['Código Distribuidor'])].head

In [5]:
df1.sample(3)

,Periodo,COD_DISTRI,COD_CLIENTE,Visualizaron,Aceptaron,%Aceptaron,Completaron,%Completaron
3159,2022-10-01,369846,6589,1,1,1.0,0,0.0
6711,2022-10-01,328660,1255,0,0,0.0,0,0.0
7482,2022-10-01,70425,1601,1,1,1.0,0,0.0


In [6]:
print("""DELETE FROM BEES_Desafios
    WHERE Periodo = {}""".format("'"+periodo_query+"'"))

DELETE FROM BEES_Desafios
    WHERE Periodo = '2022-10-01'


In [7]:
# parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
#                                      "Server=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
#                                      "DATABASE=DW_Central;"
#                                      "UID=sa;"
#                                      "PWD=DWpln21")

def run():
        # try:
        #         engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
        #         print("Conect succesfull")
        # except Exception:
        #         print("Errora in DataBase conection")
        try:
                df1.to_excel('y:/19-POWER BI/01-Total Bebidas/TACO GQ/Desafios_{}_procesado.xlsx'.format(nombre_mes), index=False)
                df1.to_sql('BEES_Desafios', con=enginex, index=False, if_exists='append', dtype={'Periodo': DATE})
                print('Carga Exitosa ' + datetime.now().strftime('%A, %d of %B %Y a las %I:%M:%S'))
        except Exception:
                print("Error al cargar los datos")
text=("""DELETE FROM BEES_Desafios
    WHERE Periodo = {}""".format("'"+periodo_query+"'"))
enginex.execute(text)
run()


Carga Exitosa Wednesday, 19 of October 2022 a las 08:00:14
